In [ ]:
# Installing packages

!pip install langchain openai

# LangChain Prompt Template

```markdown
BasePromptTemplate --> PipelinePromptTemplate
                       StringPromptTemplate --> PromptTemplate
                                                FewShotPromptTemplate
                                                FewShotPromptWithTemplates
                       BaseChatPromptTemplate --> AutoGPTPrompt
                                                  ChatPromptTemplate --> AgentScratchPadChatPromptTemplate



BaseMessagePromptTemplate --> MessagesPlaceholder
                              BaseStringMessagePromptTemplate --> ChatMessagePromptTemplate
                                                                  HumanMessagePromptTemplate
                                                                  AIMessagePromptTemplate
                                                                  SystemMessagePromptTemplate

PromptValue --> StringPromptValue
                ChatPromptValue

```

## PromptTemplate


In [2]:
import langchain
from langchain.prompts.prompt import PromptTemplate

In [3]:
my_template = PromptTemplate(
    input_variables= ['cuisine'],
    template="i want to open a {cuisine} restaurant. sugget me a good name"
)

print(my_template)

input_variables=['cuisine'] template='i want to open a {cuisine} restaurant. sugget me a good name'


In [4]:
PromptTemplate.format(my_template, cuisine="india")

'i want to open a india restaurant. sugget me a good name'

In [5]:
my_template_2 = PromptTemplate(
    input_variables= ['cuisine', 'location'],
    template= "i want to open a {cuisine} restaurant in {location}. suggest me a good name"
)
print(PromptTemplate.format(my_template_2, cuisine="Mexican", location="Vizag"))

i want to open a Mexican restaurant in Vizag. suggest me a good name


## FewShotPromptTemplate

In [6]:
# class langchain.prompts.few_shot.FewShotPromptTemplate

from langchain.prompts.few_shot import FewShotPromptTemplate

In [7]:
examples = [
    {   "question": "When was the founder of craigslist born?",
        "answer": "December 6, 1952"
     },
    {
        "question": "Who was the maternal grandfather of George Washington?",
        "answer": "Joseph Ball"
    },
    {
        "question": "Are both the directors of Jaws and Casino Royale from the same country?",
        "answer": "No"
    }
]

qa_template = PromptTemplate(
    input_variables= ["question", "answer"],
    template= "Question: {question}\nAnswer: {answer}"
)


few_shot_template = FewShotPromptTemplate(
    input_variables= ["question"],
    example_prompt = qa_template,
    prefix = "please answer the following question based on these examples:\n\nQuestion: {question}",
    suffix = "\n\nAnswer: ",
    examples= examples
)

my_prompt = few_shot_template.format(question="who is the prime minister of india?")

print(my_prompt)

please answer the following question based on these examples:

Question: who is the prime minister of india?

Question: When was the founder of craigslist born?
Answer: December 6, 1952

Question: Who was the maternal grandfather of George Washington?
Answer: Joseph Ball

Question: Are both the directors of Jaws and Casino Royale from the same country?
Answer: No



Answer: 


In [8]:
import openai
from langchain.llms import OpenAI
from secret_key import openapi_key, serp_key, huggingface_key
import os
os.environ['OPENAI_API_KEY'] = openapi_key
os.environ['HUGGINGFACEHUB_API_TOKEN'] = huggingface_key

In [9]:
llm = OpenAI(verbose=True)
name = llm(my_prompt)
print(name)

 Narendra Modi


## FewShotPromptWithTemplates

I couldnt find much difference between the **FewShotPromptTemplates** and FewShotPromptWithTemplates if you found it just let me know or fork the branch and create a pull request i will merge it. Thank you.

## AutoGPT

In [81]:
# Under Construction......!!!!!!!!!!!!


# Chains

Chains encode a sequence of calls to components like models, document retrievers, other Chains, etc.

In [10]:
from langchain.chains import LLMChain
llm = OpenAI()
name = llm("I want to open a restaurant for indian food. suggest me a fancy name for this")
print(name)



Spice Bazaar


In [37]:
my_template = PromptTemplate(
    input_variables=['cuisine', 'location'],
    template="I want to open a restaurant for {cuisine} food at {location}. suggest me a fancy name for it"
)

my_chain = LLMChain(llm=llm,prompt=my_template, verbose=True)
print(my_chain.predict(cuisine="india", location="Vizag"))



> Entering new LLMChain chain...
Prompt after formatting:
I want to open a restaurant for india food at Vizag. suggest me a fancy name for it

> Finished chain.


Mumbai Masala  


In [38]:
name_prompt = PromptTemplate(
    input_variables=["cuisine"],
    template="I want to open a restaurant for {cuisine} food. suggest me a fancy name for it."
)

menu_prompt = PromptTemplate(
    input_variables=['restaurant_name'],
    template = "suggest me some menu items for {restaurant_name}."
)

name_chain = LLMChain(llm = OpenAI(), prompt = name_prompt, verbose=True)
menu_chain = LLMChain(llm = OpenAI(), prompt = menu_prompt, verbose = True)


In [35]:
from langchain.chains import SimpleSequentialChain

s_chain = SimpleSequentialChain(chains=[name_chain, menu_chain], verbose = True)
print(s_chain("india"))



> Entering new SimpleSequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
I want to open a restaurant for india food. suggest me a fancy name for it.

> Finished chain.


Maharaja's Palace Cuisine


> Entering new LLMChain chain...
Prompt after formatting:
suggest me some menu items for 

Maharaja's Palace Cuisine.

> Finished chain.


-Tandoori Chicken
-Butter Chicken
-Lamb Rogan Josh
-Chicken Tikka Masala
-Saag Paneer
-Aloo Gobi
-Matar Paneer
-Korma
-Palak Paneer
-Kadai Paneer
-Naan Bread
-Raita
-Mango Lassi
-Gulab Jamun

> Finished chain.
{'input': 'india', 'output': '\n\n-Tandoori Chicken\n-Butter Chicken\n-Lamb Rogan Josh\n-Chicken Tikka Masala\n-Saag Paneer\n-Aloo Gobi\n-Matar Paneer\n-Korma\n-Palak Paneer\n-Kadai Paneer\n-Naan Bread\n-Raita\n-Mango Lassi\n-Gulab Jamun'}


In [31]:
from langchain.chains import SequentialChain

name_prompt = PromptTemplate(
    input_variables=["cuisine", "location"],
    template="I want to open a restaurant for {cuisine} food at {location}. suggest me a fancy name for it."
)

menu_prompt = PromptTemplate(
    input_variables=['restaurant_name'],
    template = "suggest me some menu items for {restaurant_name}."
)

name_chain = LLMChain(llm = OpenAI(), prompt = name_prompt, verbose=True, output_key="restaurant_name")
menu_chain = LLMChain(llm = OpenAI(), prompt = menu_prompt, verbose = True, output_key="menu_items")

my_restaurant = SequentialChain(chains=[name_chain, menu_chain],
                                input_variables=['cuisine', 'location'],
                                output_variables=['restaurant_name', 'menu_items'],
                                verbose=True)


print(my_restaurant({"cuisine":"indian", "location":"Vizag"}))



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
I want to open a restaurant for indian food at Vizag. suggest me a fancy name for it.

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
suggest me some menu items for 

Maharajah's Tandoori Delight.

> Finished chain.

> Finished chain.
{'cuisine': 'indian', 'location': 'Vizag', 'restaurant_name': "\n\nMaharajah's Tandoori Delight", 'menu_items': '\n\n-Tandoori Chicken\n-Naan\n-Chicken Tikka Masala\n-Lamb Rogan Josh\n-Butter Chicken\n-Vegetable Korma\n-Spicy Potato Curry\n-Chana Masala\n-Biryani\n-Saag Paneer\n-Mango Lassi\n-Gulab Jamun'}


In [82]:
from langchain.prompts import PromptTemplate
from langchain.llms import HuggingFace
from langchain.chains import LLMChain

prompt = PromptTemplate(
    input_variables=["city"],
    template="Describe a perfect day in {city}?",
)

llm = HuggingFace(
    model_name="gpt-neo-2.7B",
    temperature=0.9
)

llmchain = LLMChain(llm=llm, prompt=prompt)

llmchain.run("Paris")


ImportError: ignored